# Road Rage: Finding the Ideal Speed Limit

### Assumptions
* Drivers want to go up to 120 km/hr.
* The average car is 5 meters long.
* Drivers want at least a number of meters equal to their speed in meters/second between them and the next car.
* Drivers will accelerate 2 m/s<sup>2</sup> up to their desired speed as long as they have room to do so.
* If another car is too close, drivers will match that car's speed until they have room again.
* If a driver would hit another car by continuing, they stop.
* Drivers will randomly (10% chance each second) slow by 2 m/s.
* This section of road is one lane going one way.
* Assume that drivers enter the road at the speed they left.
* Simulation starts with 30 cars per kilometer, evenly spaced.

## Normal Mode
We have a 1 kilometer section of road being built and do not know what the speed limit should be. This notebook simulates the 1 kilometer of road. Even though this road is not circular, the simulation treats it as such in order to generate a continuous flow of traffic.

In [1]:
from copy import deepcopy
import math
import matplotlib.pyplot as plt
import numpy as np
import random
from traffic_lib import *
%matplotlib inline

In [2]:
sim1 = HighwaySim()
print(sim1.run_sim(120))

21.0696969697


In [3]:
def highway_trials(num_trials=1000):
    """
    Run num_trials of run_sim
    """
    trial_stats = []
    for _ in range(num_trials):
        sim = HighwaySim()
        trial_stats.append(sim.run_sim(120))
    return trial_stats

In [4]:
highway_100_data = highway_trials(100)

In [6]:
highway_100_mean = np.mean(np.array(highway_100_data))
highway_100_std = np.std(np.array(highway_100_data))
print("100-Trial Mean: {}".format(highway_100_mean))
print("100-Trial Standard Deviation: {}".format(highway_100_std))
print("Speed Limit Recommendation: {}".format(int(highway_100_mean + highway_100_std)))

100-Trial Mean: 18.275333333333336
100-Trial Standard Deviation: 2.3486971434611896
Speed Limit Recommendation: 20
